In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "nfl-big-data-bowl-2024"

In [2]:
library(DBI)
library(duckdb)

In [3]:
read_csv_week <- function(file_name) {
    message(file_name)
    
  readr::read_csv(file_name) %>%
    dplyr::mutate(week = str_extract(file_name, '\\d')) %>% 
    dplyr::mutate(
        x = ifelse(playDirection == 'left', 120-x, x),
        y = ifelse(playDirection == 'left', 160/3 - y, y),
        o = ifelse(playDirection == 'left', abs(o - 180), o), 
        dir = ifelse(playDirection == 'left', abs(dir - 180), dir),
        x_end = s*cos((90 - dir)*pi / 180) + x,
        y_end = s*sin((90 - dir)*pi / 180) + y,
    ) %>%
    group_by(gameId, playId) %>%
    dplyr::mutate(
        x_start = min(ifelse(club == 'football' & frameId == 1, x, NA), na.rm = TRUE)
    ) %>% 
    dplyr::ungroup() %>%
    dplyr::mutate(x = x - x_start) %>%
    dplyr::group_by(gameId, playId, frameId) %>%
    mutate(
        x_ball = min(if_else(club == 'football', x, NA), na.rm = TRUE),
        y_ball = min(if_else(club == 'football', y, NA), na.rm = TRUE),
    ) %>%
    dplyr::ungroup() %>%
    dplyr::mutate(
        dist_from_ball = sqrt( ((x - x_ball) ** 2) + ((y - y_ball) ** 2) )
    ) %>%
    janitor::clean_names()
}

project_dir <- '/kaggle/input/nfl-big-data-bowl-2024'

df_tracking <- paste0(project_dir, '/', list.files(path = project_dir, pattern = "tracking*")) %>% 
  map_dfr(read_csv_week)

/kaggle/input/nfl-big-data-bowl-2024/tracking_week_1.csv

Rows: 1407439 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (4): displayName, club, playDirection, event
dbl  (12): gameId, playId, nflId, frameId, jerseyNumber, x, y, s, a, dis, o,...
dttm  (1): time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_2.csv

Rows: 1371600 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (4): displayName, club, playDirection, event
dbl  (12): gameId, playId, nflId, frameId, jerseyNumber, x, y, s, a, dis, o,...
dttm  (1): time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
/kaggle/input/nfl-big-data-bowl-2024

In [4]:
con <- dbConnect(duckdb(), dbdir = '/kaggle/working/bdb.duckdb')
dbWriteTable(con, "tracking", df_tracking, overwrite = TRUE)
message("Table Updated")

Table Updated



In [5]:
players <- read_csv('/kaggle/input/nfl-big-data-bowl-2024/players.csv') %>% janitor::clean_names()
games <- read_csv('/kaggle/input/nfl-big-data-bowl-2024/games.csv') %>% janitor::clean_names()
plays <- read_csv('/kaggle/input/nfl-big-data-bowl-2024/plays.csv') %>% janitor::clean_names()
tackles <- read_csv('/kaggle/input/nfl-big-data-bowl-2024/tackles.csv') %>% janitor::clean_names()

dbWriteTable(con, "players", players, overwrite = TRUE)
dbWriteTable(con, "games", games, overwrite = TRUE)
dbWriteTable(con, "plays", plays, overwrite = TRUE)
dbWriteTable(con, "tackles", tackles, overwrite = TRUE)

Rows: 1683 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): height, birthDate, collegeName, position, displayName
dbl (2): nflId, weight

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 136 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): gameDate, homeTeamAbbr, visitorTeamAbbr
dbl  (5): gameId, season, week, homeFinalScore, visitorFinalScore
time (1): gameTimeEastern

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 12486 Columns: 35
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (10): ballCarrierDisplayName, playDescription, possessionTeam, defensiv...
dbl  (24): gameId, playId, 